In [1]:
import csv
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow as tf
import tensorflow_hub as hub

import requests
from PIL import Image
from io import BytesIO
import numpy as np
import pandas as pd

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [2]:
from PIL import Image, ImageDraw

import json

def vector_to_numpy(drawing, side=256):
    image = vector_to_image(drawing, side)
    image_array = np.array(image)
    return image_array

def vector_to_image(drawing, side=256):
    drawing = json.loads(drawing)
    min_x, min_y, max_x, max_y = calculate_bounding_box(drawing)

    # Calculate the offset to center the drawing within the canvas
    offset_x = (side - (max_x - min_x + 1)) // 2
    offset_y = (side - (max_y - min_y + 1)) // 2

    image = Image.new('L', (side, side), color='white')  # Create a white canvas
    draw = ImageDraw.Draw(image)

    for x, y in drawing:
        xy = [(x0 - min_x + offset_x, y0 - min_y + offset_y) for x0, y0 in zip(x, y)]
        draw.line(xy, fill='black', width=1)

    return image

def calculate_bounding_box(drawing):
    all_x = [x for x, _ in drawing]
    all_y = [y for _, y in drawing]

    min_x = min(min(x) for x in all_x)
    min_y = min(min(y) for y in all_y)
    max_x = max(max(x) for x in all_x)
    max_y = max(max(y) for y in all_y)

    return min_x, min_y, max_x, max_y

In [3]:
df = pd.DataFrame()

In [4]:
length = 432609

In [5]:
def get_image_from_file(file_path):
    images = []
    labels = []
    with open(file_path, 'r') as f:
        csv_reader = csv.reader(f)
        next(csv_reader)
        i=0
        for i in range(length//100):
            row = next(csv_reader)  
            drawing,label = row[0],row[2]
            image = vector_to_numpy(drawing)
            images.append(image)
            labels.append(label)
    return images,labels
            

In [6]:
def reshape_image(image):
    image = image.reshape(256,256,1)
    image = tf.image.resize_with_pad(image, *image_size)
    return image

In [17]:
df['image'],df['word'] = get_image_from_file('train.csv')

In [18]:
df['image'] = df['image']/255

In [19]:
model_name = "efficientnetv2-b2"
model_handle = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b2/classification/2"
image_size = (260, 260)

In [20]:
classifier = hub.load(model_handle)

In [21]:
df['image'] = df['image'].apply(reshape_image)

In [ ]:
df['image'] = df['image'].apply(tf.convert_to_tensor,args= (tf.float32,))

In [ ]:
df['image'] = df['image'][0]


In [24]:
labels_file = "https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt"

#download labels and creates a maps
downloaded_file = tf.keras.utils.get_file("labels.txt", origin=labels_file)

classes = []

with open(downloaded_file) as f:
    labels = f.readlines()
    classes = [l.strip() for l in labels]

In [25]:
probabilities = tf.nn.softmax(classifier(df['image'][0],False,False)).numpy()

top_5 = tf.argsort(probabilities, axis=-1, direction="DESCENDING")[0][:5].numpy()
np_classes = np.array(classes)

ValueError: Could not find matching concrete function to call loaded from the SavedModel. Got:
  Positional arguments (3 total):
    * <tf.Tensor 'inputs:0' shape=(260, 260, 1) dtype=float32>
    * False
    * False
  Keyword arguments: {}

 Expected these arguments to match one of the following 4 option(s):

Option 1:
  Positional arguments (3 total):
    * TensorSpec(shape=(None, None, None, 3), dtype=tf.float32, name='input_1')
    * False
    * False
  Keyword arguments: {}

Option 2:
  Positional arguments (3 total):
    * TensorSpec(shape=(None, None, None, 3), dtype=tf.float32, name='inputs')
    * False
    * False
  Keyword arguments: {}

Option 3:
  Positional arguments (3 total):
    * TensorSpec(shape=(None, None, None, 3), dtype=tf.float32, name='inputs')
    * True
    * False
  Keyword arguments: {}

Option 4:
  Positional arguments (3 total):
    * TensorSpec(shape=(None, None, None, 3), dtype=tf.float32, name='input_1')
    * True
    * False
  Keyword arguments: {}